In [ ]:
!pip install -q youtube-transcript-api
!pip install -q chromadb
!pip install -q sentence-transformers
!pip install -q scikit-learn
!pip install -q numpy

# Para usar Gemma con Transformers (alternativa a Ollama)
!pip install -q transformers
!pip install -q torch
!pip install -q accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 33.5 MB/s eta 

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas correctamente")
print(f"PyTorch disponible: {torch.cuda.is_available()}")
print(f"GPU disponible: {'Sí' if torch.cuda.is_available() else 'No (usando CPU)'}")

Librerías importadas correctamente
PyTorch disponible: False
GPU disponible: No (usando CPU)


In [ ]:
VIDEO_ID = "jNQXAC9IVRw"  # Video de ejemplo
COLLECTION_NAME = "youtube_transcripts"

# Configurar ChromaDB
print("🔧 Configurando ChromaDB...")
chroma_client = chromadb.Client()

# Función de embedding
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

print(f"Video a procesar: https://youtube.com/watch?v={VIDEO_ID}")
print(f"Nombre de colección: {COLLECTION_NAME}")

🔧 Configurando ChromaDB...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Configuración completada
Video a procesar: https://youtube.com/watch?v=jNQXAC9IVRw
Nombre de colección: youtube_transcripts


In [ ]:
# EXTRAER TRANSCRIPCIÓN DE YOUTUBE

print(f"transcripción del video: {VIDEO_ID}")

try:
    # transcripción
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(VIDEO_ID)
    print(f"Transcripción: {len(transcript)} segmentos")

    # Extraer los textos y metadatos
    texts = []
    metadatas = []

    for i, entry in enumerate(transcript):
        texts.append(entry.text)
        metadatas.append({
            'start': entry.start,
            'duration': entry.duration,
            'order': i,
            'video_id': VIDEO_ID
        })

    # Resultados
    print(f"Textos extraídos: {len(texts)} segmentos")
    print(f"Metadatos: {len(metadatas)} entradas")

    # Ejemplos
    print("\nPrimeros 3 segmentos:")
    for i in range(min(3, len(texts))):
        timestamp = metadatas[i]['start']
        print(f"[{timestamp:.1f}s] {texts[i]}")

except Exception as e:
    print(f" Error de transcripción: {e}")
    print(" Verifica que el video tenga subtítulos disponibles")

transcripción del video: jNQXAC9IVRw
Transcripción: 6 segmentos
Textos extraídos: 6 segmentos
Metadatos: 6 entradas

Primeros 3 segmentos:
[1.2s] All right, so here we are, in front of the
elephants
[5.3s] the cool thing about these guys is that they
have really...
[8.0s] really really long trunks


In [ ]:
# CHROMADB

print("base de datos vectorial")

try:
    # Crear colección
    try:
        chroma_client.delete_collection(name=COLLECTION_NAME)
        print(f" Colección existente '{COLLECTION_NAME}' eliminada")
    except:
        pass

    # Crear nueva colección
    collection = chroma_client.create_collection(
        name=COLLECTION_NAME,
        embedding_function=sentence_transformer_ef
    )
    print(f"Colección '{COLLECTION_NAME}' creada")

    ids = [f"{VIDEO_ID}_{i}" for i in range(len(texts))]

    # Añadir documentos a la colección
    print("Añadiendo documentos a ChromaDB")
    collection.add(
        documents=texts,
        metadatas=metadatas,
        ids=ids
    )

    print(f"{len(texts)} documentos añadidos")

    # Ejemplo para probar
    print("\n búsqueda de ejemplo")
    test_results = collection.query(
        query_texts=["tema principal"],
        n_results=2
    )

    print("Documentos encontrados en la prueba:")
    for i, doc in enumerate(test_results['documents'][0]):
        metadata = test_results['metadatas'][0][i]
        print(f"[{metadata['start']:.1f}s] {doc[:100]}...")

except Exception as e:
    print(f"Error creando la base de datos: {e}")

base de datos vectorial...
Colección 'youtube_transcripts' creada
Añadiendo documentos a ChromaDB
6 documentos añadidos

 búsqueda de ejemplo
Documentos encontrados en la prueba:
[8.0s] really really long trunks...
[5.3s] the cool thing about these guys is that they
have really......


In [ ]:
# Analisis con ML

try:
    # Cargar el modelo de embeddings
    print("modelo de embeddings")
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Generara embeddings para todas las oraciones
    sentence_embeddings = embedder.encode(texts, show_progress_bar=True)
    print(f"Embeddings generados: {sentence_embeddings.shape}")

    # etiquetas simuladas
    # Oraciones más largas son consideradas más importantes
    y = np.zeros(len(texts))
    for i, text in enumerate(texts):
        if len(text.split()) > 4:
            y[i] = 1

    X_train, X_test, y_train, y_test = train_test_split(
        sentence_embeddings, y, test_size=0.2, random_state=42
    )

    # estadísticas
    print(f"\n Estadísticas del dataset:")
    print(f"Total de segmentos: {len(texts)}")
    print(f"Entrenamiento: {len(X_train)} segmentos")
    print(f"Prueba: {len(X_test)} segmentos")
    print(f"Oraciones importantes en entrenamiento: {int(np.sum(y_train))}")
    print(f"Oraciones importantes en prueba: {int(np.sum(y_test))}")
    print(f"Porcentaje de importancia: {np.mean(y)*100:.1f}%")

    # ejemplos
    print(f"\n Ejemplos de oraciones consideradas importantes:")
    important_indices = np.where(y == 1)[0][:5]
    for idx in important_indices:
        timestamp = metadatas[idx]['start']
        print(f"[{timestamp:.1f}s] {texts[idx]}")

except Exception as e:
    print(f" Error en análisis ML: {e}")

modelo de embeddings


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generados: (6, 384)

 Estadísticas del dataset:
Total de segmentos: 6
Entrenamiento: 4 segmentos
Prueba: 2 segmentos
Oraciones importantes en entrenamiento: 1
Oraciones importantes en prueba: 2
Porcentaje de importancia: 50.0%

 Ejemplos de oraciones consideradas importantes:
[1.2s] All right, so here we are, in front of the
elephants
[5.3s] the cool thing about these guys is that they
have really...
[16.9s] and that's pretty much all there is to
say


In [ ]:
# CMODELO GEMMA CON LLAMA-CPP-PYTHON

!pip install -q llama-cpp-python
!pip install -q huggingface_hub

print(" Cargando modelo Gemma con llama-cpp-python...")

from huggingface_hub import login, hf_hub_download
from llama_cpp import Llama
import os

# modelo
REPO_ID = "microsoft/Phi-3-mini-4k-instruct-gguf"
GGUF_FILE = "Phi-3-mini-4k-instruct-q4.gguf"

#REPO_ID = "google/gemma-2b-it-GGUF"
#GGUF_FILE = "gemma-2b-it.q4_0.gguf"

try:
    print(" Iniciando sesión en Hugging Face...")

    print(f"Descargando modelo: {REPO_ID}")
    print(f"Archivo: {GGUF_FILE}")

    model_path = hf_hub_download(
        repo_id=REPO_ID,
        filename=GGUF_FILE,
        local_dir="./models",
        local_dir_use_symlinks=False
    )

    print(f"Modelo descargado en: {model_path}")


    print("Inicializar modelo.")
    llm = Llama(
        model_path=model_path,
        n_ctx=2048,
        n_threads=2,
        n_gpu_layers=0,
        verbose=False
    )

    print("Modelo Gemma/Phi-3 cargado")

    print("Probando modelo...")
    test_response = llm.create_chat_completion(
        messages=[
            {
                "role": "user",
                "content": "Hola, ¿funcionas correctamente?"
            }
        ],
        max_tokens=50
    )

    print("Respuesta de prueba:")
    print(test_response['choices'][0]['message']['content'])

except Exception as e:
    print(f"Error cargando modelo: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
🤖 Cargando modelo Gemma con llama-cpp-python...
🔑 Iniciando sesión en Hugging Face...
Descargando modelo: microsoft/Phi-3-mini-4k-instruct-gguf
Archivo: Phi-3-mini-4k-instruct-q4.gguf


Phi-3-mini-4k-instruct-q4.gguf:   0%|          | 0.00/2.39G [00:00<?, ?B/s]

Modelo descargado en: models/Phi-3-mini-4k-instruct-q4.gguf
Inicializar modelo.


llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


Modelo Gemma/Phi-3 cargado
Probando modelo...
Respuesta de prueba:
 Sí, claro que sí. Estoy funcionando correctamente. Como asistente de inteligencia artificial, estoy aquí para ayudarte a responder preguntas, ofrecer información y asistirte en


In [ ]:
# RAG

def retrieve_context(query, n_results=3):
    """
    Recupera contexto relevante de ChromaDB basado en la consulta
    """
    try:
        results = collection.query(
            query_texts=[query],
            n_results=n_results
        )

        context_parts = []
        sources = []

        for i, doc in enumerate(results['documents'][0]):
            metadata = results['metadatas'][0][i]
            timestamp = f"[{metadata['start']:.1f}s]"
            context_parts.append(f"{timestamp} {doc}")
            sources.append(metadata)

        context = "\n".join(context_parts)
        return context, sources

    except Exception as e:
        print(f"Error en recuperación: {e}")
        return "", []

def generate_response_with_llama(query, context):
    """
    Genera respuesta usando llama-cpp-python con el contexto recuperado
    """
    try:
        prompt = f"""Eres un asistente experto que analiza transcripciones de videos de YouTube.

TRANSCRIPCIÓN DEL VIDEO:
{context}

PREGUNTA DEL USUARIO: {query}

INSTRUCCIONES:
- Responde basándote únicamente en la transcripción proporcionada
- Si no tienes información suficiente, dilo claramente
- Incluye referencias a los timestamps [XXs] cuando sea relevante
- Sé conciso pero completo
- Responde en español

RESPUESTA:"""

        print("Generando respuesta con llama-cpp...")

        response = llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": "Eres un asistente experto en análisis de videos que responde basándose únicamente en transcripciones proporcionadas."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            max_tokens=400,
            temperature=0.7,
            top_p=0.9,
            stop=["PREGUNTA:", "TRANSCRIPCIÓN:"],
        )

        answer = response['choices'][0]['message']['content'].strip()

        return answer

    except Exception as e:
        return f"Error generando respuesta: {e}"

def query_video_llama(query, n_results=3):
    """
    Función principal que combina recuperación y generación con llama-cpp
    """
    print(f"Procesando consulta: '{query}'")

    # contexto relevante
    print("Buscando información")
    context, sources = retrieve_context(query, n_results)

    if not context:
        return "No se pudo recuperar contexto relevante"

    print(f"Encontradas {len(sources)} fuentes relevantes")

    # Generar respuesta
    response = generate_response_with_llama(query, context)

    return {
        'answer': response,
        'context': context,
        'sources': sources
    }

# pruebas rápidas
def simple_chat(query):
    """
    Chat directo sin RAG para probar el modelo
    """
    try:
        response = llm.create_chat_completion(
            messages=[
                {
                    "role": "user",
                    "content": query
                }
            ],
            max_tokens=200,
            temperature=0.7
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Error: {e}"

print("Funciones RAG configuradas")

# Prueba rápida del sistema
print("\n conexión con el modelo")
test_msg = simple_chat("Hola, ¿puedes ayudarme a analizar videos?")
print(f"Respuesta de prueba: {test_msg}")

Funciones RAG configuradas

 conexión con el modelo
Respuesta de prueba:  Yes, I can help you analyze videos, but my capabilities are based on processing and understanding text-based information. For video analysis, I can guide you through interpreting video content, provide insights on video quality, and help you understand the context or information presented in a video. However, direct video processing would require specialized software or APIs. If you have specific aspects of a video you'd like to analyze, such as themes, content, or sentiment, feel free to describe it, and I can assist you in discussing those elements.


In [ ]:

def display_result(result):
    """Función helper para mostrar resultados de forma organizada"""
    print("="*70)
    print("RESPUESTA DEL SISTEMA:")
    print("="*70)
    print(result['answer'])
    print("\n" + "="*70)
    print("CONTEXTO UTILIZADO:")
    print("="*70)
    print(result['context'])
    print("\n" + "="*70)
    print("FUENTES (TIMESTAMPS):")
    print("="*70)
    for source in result['sources']:
        minutes = int(source['start'] // 60)
        seconds = int(source['start'] % 60)
        print(f"- Minuto {minutes}:{seconds:02d} (orden: {source['order']})")
    print("="*70)

def ask_question_llama(question):
    """
    Función optimizada para hacer preguntas al sistema con llama-cpp
    Úsala así: ask_question_llama("Tu pregunta aquí")
    """
    if not question.strip():
        return "Por favor ingresa una pregunta"

    try:
        result = query_video_llama(question)

        print(f"\nPREGUNTA: {question}")
        print("="*50)
        print("RESPUESTA:")
        print(result['answer'])
        print("\nTIMESTAMPS RELEVANTES:")
        for source in result['sources']:
            minutes = int(source['start'] // 60)
            seconds = int(source['start'] % 60)
            print(f"- {minutes}:{seconds:02d}")
        print("="*50)

        return result

    except Exception as e:
        return f"Error: {e}"

# Configuración final del sistema
print("¡SISTEMA RAG YOUTUBE + GEMMA/PHI-3 LISTO!")
print("="*70)
print("Video analizado:", f"https://youtube.com/watch?v={VIDEO_ID}")
print("Modelo:", "llama-cpp-python con GGUF")
print("Segmentos procesados:", len(texts))
print("="*70)
print("Puedes hacer preguntas como:")
print("- ¿De qué trata el video?")
print("- ¿Cuáles son los puntos principales?")
print("- Resume el contenido en 3 puntos")
print("- ¿Se menciona [tema específico]?")
print("- ¿Qué se dice en el minuto X?")
print("="*70)

# Prueba automática con una pregunta
print("\n PRUEBA AUTOMÁTICA:")
sample_question = "¿De qué trata principalmente este video?"
try:
    result = query_video_llama(sample_question)
    display_result(result)
except Exception as e:
    print(f" Error en prueba: {e}")

print("\n" + "="*70)
print("Sistema completamente configurado y listo!")
print("Usa ask_question_llama('tu pregunta') para consultar")
print("="*70)

# Ejemplo adicional de uso directo:
print("\n EJEMPLO DE USO DIRECTO:")
print("ask_question_llama('¿Cuáles son los puntos más importantes del video?')")

¡SISTEMA RAG YOUTUBE + GEMMA/PHI-3 LISTO!
Video analizado: https://youtube.com/watch?v=jNQXAC9IVRw
Modelo: llama-cpp-python con GGUF
Segmentos procesados: 6
Puedes hacer preguntas como:
- ¿De qué trata el video?
- ¿Cuáles son los puntos principales?
- Resume el contenido en 3 puntos
- ¿Se menciona [tema específico]?
- ¿Qué se dice en el minuto X?

 PRUEBA AUTOMÁTICA:
🔍 Procesando consulta: '¿De qué trata principalmente este video?'
Buscando información
Encontradas 3 fuentes relevantes
Generando respuesta con llama-cpp...
RESPUESTA DEL SISTEMA:
El video trata principalmente sobre algo que se menciona como "really really long trunks", lo cual puede implicar una descripción o una característica destacada de algo o alguien, aunque no se especifica claramente a partir de la transcripción proporcionada. Los timestamps [8.0s] y [5.3s] son relevantes ya que contienen la información más directa sobre el tema del video. Sin embargo, debido a la falta de contexto, no se puede determinar con preci

In [ ]:

print("INTERACCIÓN CON EL LLM")

# =============================================================================
# 1. CHAT DIRECTO (sin usar la base de datos del video)
# =============================================================================

print("\nCHAT DIRECTO CON EL LLM:")

def demo_direct_chat():
    """Ejemplos de chat directo"""
    preguntas_directas = [
        "¿Cuál es la capital de España?",
        "Explícame qué es Python en 2 líneas",
        "¿Cómo estás hoy?",
        "Dame 3 consejos para estudiar mejor"
    ]

    print("Ejemplos de preguntas directas al LLM:")
    for i, pregunta in enumerate(preguntas_directas, 1):
        print(f"\n{i}. Pregunta: {pregunta}")
        try:
            respuesta = simple_chat(pregunta)
            print(f"   Respuesta: {respuesta}")
        except:
            print(f"   Código: simple_chat('{pregunta}')")

# =============================================================================
# 2. RAG - Preguntas sobre el video
# =============================================================================

print("\nPREGUNTAS SOBRE EL VIDEO (RAG):")
print("-" * 40)

def demo_video_rag():
    """Ejemplos de preguntas sobre el video"""
    preguntas_video = [
        "¿De qué trata el video?",
        "¿Cuáles son los puntos principales?",
        "Resume el contenido en 3 puntos",
        "¿Qué se menciona en los primeros minutos?",
        "¿Hay alguna conclusión importante?"
    ]

    print("Ejemplos de preguntas sobre el video:")
    for i, pregunta in enumerate(preguntas_video, 1):
        print(f"\n{i}. Pregunta: {pregunta}")
        print(f"   Código: ask_question_llama('{pregunta}')")

# =============================================================================
# 3. FUNCIÓN MEJORADA PARA USO INTERACTIVO
# =============================================================================

def chat_inteligente(pregunta):
    """
    Función que decide automáticamente si usar RAG o chat directo
    """
    # Palabras clave que indican pregunta sobre el video
    video_keywords = [
        'video', 'transcripción', 'dice', 'menciona', 'habla',
        'trata', 'explica', 'minuto', 'segundo', 'contenido',
        'tema', 'punto', 'resume', 'resumir'
    ]

    # Convertir a minúsculas para comparar
    pregunta_lower = pregunta.lower()

    # Verificar si la pregunta es sobre el video
    es_sobre_video = any(keyword in pregunta_lower for keyword in video_keywords)

    if es_sobre_video:
        print("Detecté que preguntas sobre el video, usando RAG...")
        return ask_question_llama(pregunta)
    else:
        print("Pregunta general, usando chat directo...")
        return simple_chat(pregunta)

# =============================================================================
# 4. EJEMPLOS PRÁCTICOS
# =============================================================================

print("\n Funcion Inteligente:")
print("-" * 40)

ejemplos_mixtos = [
    "¿Qué tiempo hace hoy?",  # Chat directo
    "¿De qué trata el video?",  # RAG
    "Explícame la fotosíntesis",  # Chat directo
    "¿Qué dice el video sobre este tema?",  # RAG
]

print("Ejemplos de uso con detección automática:")
for i, pregunta in enumerate(ejemplos_mixtos, 1):
    video_keywords = ['video', 'transcripción', 'dice', 'menciona', 'habla', 'trata']
    es_video = any(k in pregunta.lower() for k in video_keywords)
    tipo = "RAG (Video)" if es_video else "Chat Directo"

    print(f"\n{i}. '{pregunta}'")
    print(f"   → Usará: {tipo}")
    print(f"   → Código: chat_inteligente('{pregunta}')")

# =============================================================================
# 5. INTERFAZ SIMPLE PARA PROBAR
# =============================================================================

print("\n" + "="*60)
print("COLAB:")
print("="*60)

codigo_ejemplo = '''
# Para chat directo:
respuesta = simple_chat("¿Cuál es la capital de Francia?")
print(respuesta)

# Para preguntas del video:
ask_question_llama("¿De qué trata el video?")

# Para uso automático:
chat_inteligente("¿Qué menciona el video sobre inteligencia artificial?")
'''

print(codigo_ejemplo)

# Ejecutar demos
demo_direct_chat()
demo_video_rag()

INTERACCIÓN CON EL LLM

CHAT DIRECTO CON EL LLM:

PREGUNTAS SOBRE EL VIDEO (RAG):
----------------------------------------

 Funcion Inteligente:
----------------------------------------
Ejemplos de uso con detección automática:

1. '¿Qué tiempo hace hoy?'
   → Usará: Chat Directo
   → Código: chat_inteligente('¿Qué tiempo hace hoy?')

2. '¿De qué trata el video?'
   → Usará: RAG (Video)
   → Código: chat_inteligente('¿De qué trata el video?')

3. 'Explícame la fotosíntesis'
   → Usará: Chat Directo
   → Código: chat_inteligente('Explícame la fotosíntesis')

4. '¿Qué dice el video sobre este tema?'
   → Usará: RAG (Video)
   → Código: chat_inteligente('¿Qué dice el video sobre este tema?')

COLAB:

# Para chat directo:
respuesta = simple_chat("¿Cuál es la capital de Francia?")
print(respuesta)

# Para preguntas del video:
ask_question_llama("¿De qué trata el video?")

# Para uso automático:
chat_inteligente("¿Qué menciona el video sobre inteligencia artificial?")

Ejemplos de preguntas